## Download Necessary Libraries

In [1]:
!pip install beautifulsoup4

In [2]:
!pip install hazm faiss-cpu tqdm

## Import Necessary Libraries

In [3]:
import kagglehub
import os
from tqdm import tqdm
from hazm import Normalizer, WordTokenizer, Stemmer, Lemmatizer, stopwords_list
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import faiss
import numpy as np

## Download the Dataset (Hamshahri-Corpus)

In [5]:
path = kagglehub.dataset_download("ehsankhani/hamshahri-corpus")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1


Paths:

In [6]:
dataset_path = "/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1"
for root, dirs, files in os.walk(dataset_path):
    for name in files:
        print(os.path.join(root, name))


/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/persian_stopwords.txt
/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/HamshahriCorpus/2005/HAM2-840706-147.ham
/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/HamshahriCorpus/2005/HAM2-840325-107.ham
/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/HamshahriCorpus/2005/HAM2-840910-056.ham
/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/HamshahriCorpus/2005/HAM2-840308-038.ham
/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/HamshahriCorpus/2005/HAM2-840808-169.ham
/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/HamshahriCorpus/2005/HAM2-840808-165.ham
/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/HamshahriCorpus/2005/HAM2-840814-048.ham
/root/.cach

Print a file's content:

In [7]:
sample_file_path = "/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/HamshahriCorpus/2003/HAM2-811011-027.ham"

# Read and parse as plain text
with open(sample_file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Clean lines
lines = [line.strip() for line in lines if line.strip()]

# Separate title and text
title = lines[0] if lines else "No Title"
text = "\n".join(lines[1:]) if len(lines) > 1 else "No Text"

print("Title:\n", title)
print("\nText:\n", text[:1000])


Title:
 آغاز ساخت بدنه و سرريز بلندترين سد

Text:
 كشور
عمليات اجرائي ساخت بدنه و سرريز سد كارون 4 به عنوان بلندترين سد بتني در دست ساخت
كشور شروع شد.
در اين مراسم كه تني چند از نمايندگان مردم استان چهارمحال و بختياري در مجلس شوراي
اسلامي، مديران سازمان توانير و قائم مقام وزير نيرو شركت داشتند، اولين انفجارها براي
آماده سازي محل ساخت بدنه سد و سرريز بزرگ كارون 4 آغاز شد.
سد كارون 4 يكي از سلسله سدهاي زنجيره اي طراحي شده بر روي رودخانه كارون است كه
عمليات احداث سيستم انحراف آب آن از سال 76 شروع شده و براي احداث تونل انحراف اين طرح
كه آب رودخانه كارون را از مسير اصلي خود منحرف و 1500 متر مسير رودخانه را براي ساخت
سد خشك مي كند بالغ بر 140 هزار مترمكعب حفاري و 70 هزارمتر مكعب بتن ريزي صورت گرفته
است.


## Convert the Raw Dataset to Lists of Titles and Texts

In [8]:
base_path = "/root/.cache/kagglehub/datasets/ehsankhani/hamshahri-corpus/versions/1/HamshahriData/HamshahriCorpus/2003"

titles, raw_texts = [], []

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".ham"):
            file_path = os.path.join(root, file)
            with open(file_path, "r", encoding="utf-8") as f:
                lines = [line.strip() for line in f if line.strip()]
                if len(lines) > 1:
                    titles.append(lines[0])
                    raw_texts.append("\n".join(lines[1:]))


In [9]:
print(titles)

['كوتاه از سياست جهان', 'حاشيه ايران', 'نگاهي به وضعيت فيلم كوتاه در آستانه برگزاري جشنواره كارگردانان جوان', 'سايه روشن اقتصاد', 'در هفته اول شهريور ماه برگزار مي\xa0شود', 'وضعيت كلان بخش كشاورزي(2)', 'صندوق بين\xa0المللي پول:', 'خزر هر روز آلوده\xa0تر از ديروز', 'دبير ستاره', 'خط پايان', 'سخن نخست', 'بسكتبال با ويلچر قهرماني ايران در گام آخر مقدماتي', '70 درصدتهراني ها به استحكام ساختمان مسكوني خود اطمينان ندارند', 'همايون شاهرخي: من يك شغله هستم', 'نگاه امروز', 'سايه\xa0روشن ورزش', 'ديدگاه', 'پنج ميليارد دلار از محصولات پتروشيمي پيش\xa0فروش شده است', 'مهمترين كاركرد جنگل ها توليد اكسيژن است', 'مديركل بازرسي آموزش و پرورش:', 'نگاه', 'بحران عراق و ادامه رايزني ها در تهران', 'با احتساب صادرات بازارچه\xa0اي و چمداني', 'شمار قربانيان تصادفات جاده\xa0اي همچنان رو به افزايش است', 'براي پنجاه و پنجمين بار برگزار مي شود', 'مدير مسئول ايران به دادگاه كاركنان دولت احضار شد', 'خو ن لا\xa0زم داريم', 'سايه\xa0روشن ورزش', 'مدير كل امور صادرات گمرك پيش بيني مي كند:', 'در حاشيه خبرها', 'توسط دبير ست

## Preprocess the dataset using Hazm

In [10]:
normalizer = Normalizer()
tokenizer = WordTokenizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()
stop_words = set(stopwords_list())

In [11]:
def preprocess(text):
    text = normalizer.normalize(text)
    tokens = tokenizer.tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [lemmatizer.lemmatize(stemmer.stem(t)) for t in tokens]
    return " ".join(tokens)

In [12]:
# پیش‌پردازش تمام متون
documents = [preprocess(text) for text in tqdm(raw_texts)]

100%|██████████| 1076/1076 [00:06<00:00, 159.51it/s]


## Define Search Method Based on Cosine Similarity

In [13]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

In [14]:
def search(query, top_k=5):
    query_vector = vectorizer.transform([preprocess(query)])
    similarity = cosine_similarity(query_vector, tfidf_matrix).flatten()
    results = similarity.argsort()[::-1][:top_k]
    return [(titles[i], raw_texts[i], similarity[i]) for i in results]


## Improve the System and Define Fast Search Method

In [15]:
faiss_index = faiss.IndexFlatIP(tfidf_matrix.shape[1])
faiss_index.add(tfidf_matrix.toarray())

In [16]:
def fast_search(query, top_k=5):
    query_vector = vectorizer.transform([preprocess(query)]).toarray()
    scores, indices = faiss_index.search(query_vector, top_k)
    return [(titles[i], raw_texts[i], scores[0][j]) for j, i in enumerate(indices[0])]


## Test the System with some queries

Query 1:

In [17]:
query = "سد کارون"
results = fast_search(query)

for i, (title, text, score) in enumerate(results):
    print(f"\nResult {i+1} (Similarity: {score:.4f})")
    print(f"{title}\n{text[:500]}...\n")



Result 1 (Similarity: 0.6538)
آغاز ساخت بدنه و سرريز بلندترين سد
كشور
عمليات اجرائي ساخت بدنه و سرريز سد كارون 4 به عنوان بلندترين سد بتني در دست ساخت
كشور شروع شد.
در اين مراسم كه تني چند از نمايندگان مردم استان چهارمحال و بختياري در مجلس شوراي
اسلامي، مديران سازمان توانير و قائم مقام وزير نيرو شركت داشتند، اولين انفجارها براي
آماده سازي محل ساخت بدنه سد و سرريز بزرگ كارون 4 آغاز شد.
سد كارون 4 يكي از سلسله سدهاي زنجيره اي طراحي شده بر روي رودخانه كارون است كه
عمليات احداث سيستم انحراف آب آن از سال 76 شروع شده و براي احداث تونل انحراف اين طرح
كه آب رودخانه ك...


Result 2 (Similarity: 0.4024)
پايان عصر سدهاي بزرگ
از سال 1930 ميلادي كه احداث سدهاي بزرگ آغاز شده است حداقل 30 ميليون نفر به علت به زيرآب رفتن زمين هاي كشاورزي و آبادي ها، آواره  و يا ناچار به مهاجرت شده اند
براساس آخرين آمار كميسيون بين المللي سدهاي بزرگ حدود 40000 سد بزرگ بر روي رودخانه هاي جهان ساخته شده است. بيشتر آنها درعرض 35 سال اخير ساخته شده اند و چيزي بيش از 1600 سد ديگر در 40 كشور جهان در حال ساخت است. اما آياعصر

Query 2:

In [18]:
query = "زلزله"
results = fast_search(query)

for i, (title, text, score) in enumerate(results):
    print(f"\nResult {i+1} (Similarity: {score:.4f})")
    print(f"{title}\n{text[:500]}...\n")


Result 1 (Similarity: 0.6154)
وزير بهداشت خبر داد
70 هزار كشته و مجروح
65 تا 70 درصد مردم شهر بم كشته يا مجروح شده اند
بم - خبرنگار اعزامي همشهري - شهبازي: در پي وقوع زلزله 3/6 ريشتر بامداد جمعه در شهرستان بم، مردم اين شهر نخستين شب پس از حادثه را در تاريكي و اندوهي جانكاه به صبح رساندند.
صداي ناله دلخراش زلزله زدگان بخاطر غم از دست دادگانشان و شدت جراحات وارده از ميان آوارها شنيده مي شود. اين در حالي است كه براساس گزارش مقامات رسمي هر لحظه بر آمار كشته شدگان افزوده مي شود. وزير بهداشت اعلام كرد: 70 هزار نفر در شهر بم كشته و يا مجروح شده اند وي...


Result 2 (Similarity: 0.5883)
تجمع انرژي در گسل هاي فعال تهران
مقاوم سازي مهمترين راه مقابله با زلزله است
دكتر محسن  غفوري  آشتياني  رئيس  پژوهشگاه  بين المللي  زلزله شناسي  ومهندسي  زلزله  در مصاحبه  با واحد مركزي  خبر با اعلام  اين  خبر افزود: گسل  هاي  اطراف  تهران  گسل  هاي  فعالي  هستند و هم  اكنون  نيز درحال  تجمع  انرژي  هستند.
وي احتمال وقوع  زلزله  در تهران  را جدي خواند و با انتقاد از ساخت  و سازهاي  غيراصولي  در تهران  گفت : بنا

Query 3:

In [19]:
query = "اقتصاد کلان"
results = fast_search(query)

for i, (title, text, score) in enumerate(results):
    print(f"\nResult {i+1} (Similarity: {score:.4f})")
    print(f"{title}\n{text[:500]}...\n")



Result 1 (Similarity: 0.2518)
نگاهي به نقش تاريخي گمرك در تسهيل اقتصاد تجاري ايران
گمرك: مأموريم و معذور
* در جهاني كه به سمت اتحاد اقتصادي پيش مي رود حصر گمرك در ميان سياست هاي اقتصادي متكي بر دولت و جريانات مدعي اقتصاد مي تواند عواقب بسيار ناگواري داشته باشد * گمرك مجبور است تن به اجراي سياست هايي بدهد كه خود هيچ اعتقادي به آنها ندارد. سياست هايي كه هنوز هم بوي كهنگي مفاهيم اقتصاد بسته را مي دهند
علي دهقان
ديگر امروز نظريه پردازان تجارت جهاني هم نمي توانند پا به پاي تحولات اقتصاد جهاني حركت كنند. شايد اين جمله «اكونوميست» كه «روش هاي تجارت بين الملل با سرعتي نه چندان كمتر از...


Result 2 (Similarity: 0.2376)
مشكل اقتصاد ما سياسي است
تحليل عزت الله سحابي از اقتصاد _ ايران 1
زهرا ابراهيمي
مصرف سوخت ايران 16 برابر كشورهاي صنعتي اروپاو ژاپن است اين يعني ريخت وپاش كه
علاوه بر دولت مردم هم مقصر هستند
كره به اين دليل كره شد كه وارد بازار جهاني شد و هيچ امتيازي به كالاهاي داخلي خودش
نداد رقيب ژاپن، اروپا و آمريكا شد ژاپن كه كشتي هاي نفت كش 500 هزارتني را مي ساخت
الان سفارشات جهاني را مي گي

Query 4:

In [21]:
query = "بورس تهران"
results = fast_search(query)

for i, (title, text, score) in enumerate(results):
    print(f"\nResult {i+1} (Similarity: {score:.4f})")
    print(f"{title}\n{text[:500]}...\n")



Result 1 (Similarity: 0.6153)
آگاهانه در بورس سهام بخريد
مسئول  شكايات  بورس  اوراق  بهادار تهران  گفت : از ابتداي  آبان  ماه  سال جاري  تاكنون ، 114 مورد شكايت  به  بورس  اوراق  بهادار تهران  رسيده  كه  اين سازمان  پس  از بررسي  آنها، تمام  شكايات  را پاسخ  داده  است .
نادر حريري  پارسا در گفت وگو با خبرنگار ايرنا، با اشاره به  قانوني  نبودن  برخي  از شكايات  انجام  شده  توسط سهامداران  افزود: 60 درصدشكايات  بورس  تهران  به  دليل  درگيري  و عدم  توافق  بين  كارگزاران  و مشتريان بروز مي كند.
وي  بر قانوني  بودن  مبناي  شكايات  در بورس  تأكيد كرد و گفت...


Result 2 (Similarity: 0.5024)
انتقاد مديرعامل بانك سپه از ايجاد محدوديت براي بانكهاي كشور
گروه اقتصادي: عليرضا شيراني مديرعامل بانك سپه و عضو شوراي بورس تهران در يك كنفرانس مطبوعاتي اعلام كرد: در هيچ كشوري ديده نشده است بانكها را مجبور به دادن تسهيلات خاص كنند و اين رويه در ايران نيز بايد برداشته شود.
شيراني از اجراي مقررات يكسان و متمركز در نظام بانكي و محدود كردن مديران بانكها انتقاد كرد و گفت: ايجاد محدوديت در اعطاي تسهيلات، مسئو